In [90]:
import sys
import re, numpy as np, pandas as pd
import tqdm
import glob
from cleantext import clean
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['https_www','seems_like','do','not','imgur','tkg','https','http','could','www','com','ever','doesnt_seem',
                  'xxxx','else','would','also','ea','&amp','#x200B','oh','etc','yeah','nan','however','even','dont_know','sa',
                  "looks_like",'especially','may','sounds_like'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [91]:
# LoadDataset
df=pd.read_csv('homepod_merged.csv')
print(df.shape)
df.head(1)

(8332, 12)


,post_id,title,selftext,full_link,author,score,publish_date,num_of_comments,permalink,flair,comment_msg,content
0,6fgd2h,Welcome,Welcome!,https://www.reddit.com/r/HomePod/comments/6fgd...,ChicagoIL,12,2017-06-06 03:13:54,7,/r/HomePod/comments/6fgd2h/welcome/,NaN,"['first ones here!,Thanks!\n\nI was trying to ...","Welcome Welcome! ['first ones here!,Thanks!\n\..."


In [92]:
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI)

def preprocess_tweet(row):
    text = row['content']
    text = text.replace('r/','')
    text = p.clean(text)
    text = clean(text,     
                 fix_unicode=True,              # fix various unicode errors
                 to_ascii=True,                 # transliterate to closest ASCII representation
                 lower=True,                    # lowercase text
                 no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
                 no_urls=True,                  # replace all URLs with a special token
                 no_emails=True,                # replace all email addresses with a special token
                 no_phone_numbers=True,         # replace all phone numbers with a special token
                 no_numbers=True,               # replace all numbers with a special token
                 no_digits=True,                # replace all digits with a special token
                 no_currency_symbols=True,      # replace all currency symbols with a special token
                 no_punct=True,                 # remove punctuations
                 lang="en",                     # set to 'de' for German special handling
                 replace_with_punct="",          # instead of removing punctuations you may replace them
                 replace_with_url="",
                 replace_with_email="",
                 replace_with_phone_number="",
                 replace_with_number="",
                 replace_with_digit="",
                 replace_with_currency_symbol=""
                )
    text = text.replace('amp','')
    text = text.replace('nan','')
    return text

df['content'] = df.apply(preprocess_tweet, axis=1)
df

,post_id,title,selftext,full_link,author,score,publish_date,num_of_comments,permalink,flair,comment_msg,content
0,6fgd2h,Welcome,Welcome!,https://www.reddit.com/r/HomePod/comments/6fgd...,ChicagoIL,12,2017-06-06 03:13:54,7,/r/HomePod/comments/6fgd2h/welcome/,NaN,"['first ones here!,Thanks!\n\nI was trying to ...",welcome welcome first ones herethanks i was tr...
1,6fgd34,May I help with this subreddit?,NaN,https://www.reddit.com/r/HomePod/comments/6fgd...,[deleted],1,2017-06-06 03:13:58,0,/r/HomePod/comments/6fgd34/may_i_help_with_thi...,NaN,['nan'],may i help with this subreddit
2,6fggaw,First Impressions,Share your first impressions here:,https://www.reddit.com/r/HomePod/comments/6fgg...,ChicagoIL,8,2017-06-06 03:27:39,33,/r/HomePod/comments/6fggaw/first_impressions/,NaN,['Big questions for me.\n\n* Does it accept Ai...,first impressions share your first impressions...
3,6fghjj,HomePod - Article from 2004,NaN,https://www.reddit.com/r/HomePod/comments/6fgh...,HughJafro,6,2017-06-06 03:32:30,3,/r/HomePod/comments/6fghjj/homepod_article_fro...,NaN,"[""Wow...neat find!,That's crazy!""]",homepod article from wowneat findthats crazy
4,6fl3j7,Multiple users,I'm curious to know how the HomePod is going t...,https://www.reddit.com/r/HomePod/comments/6fl3...,ChubbyTio,6,2017-06-06 19:13:18,0,/r/HomePod/comments/6fl3j7/multiple_users/,NaN,"[""Yeah, it's not really that useful if it does...",multiple users im curious to know how the home...
...,...,...,...,...,...,...,...,...,...,...,...,...
8327,nokvcn,why does it look like Music is playing through...,i just picked up a second OG HomePod so that i...,https://www.reddit.com/r/HomePod/comments/nokv...,brono_trigger,1,2021-05-31 05:19:21,9,/r/HomePod/comments/nokvcn/why_does_it_look_li...,Question/Support,['If you bring the airplay menu on IOS when t...,why does it look like music is playing through...
8328,nonk7t,Homepod mini light stays on,Anyone know why the light on my mini stays on ...,https://www.reddit.com/r/HomePod/comments/nonk...,adamhodge,1,2021-05-31 07:41:55,2,/r/HomePod/comments/nonk7t/homepod_mini_light_...,Question/Support,['Is it connected to the AppleTV?'],homepod mini light stays on anyone know why th...
8329,noobbi,A HomePod + Apple TV Sound Bar?,NaN,https://www.reddit.com/r/HomePod/comments/noob...,lightsd,1,2021-05-31 08:26:01,1,/r/HomePod/comments/noobbi/a_homepod_apple_tv_...,Rumor,['I didn’t see this posted here yet. Thought i...,a homepod + apple tv sound bar i didnt see thi...
8330,nop93s,HomeTheater lets you control volume even when ...,Just did this on accident and realized that ev...,https://www.reddit.com/r/HomePod/comments/nop9...,Temporary-Employment,1,2021-05-31 09:23:17,19,/r/HomePod/comments/nop93s/hometheater_lets_yo...,Tip,"['Also,works with eARC. I can adjust ps5 volum...",hometheater lets you control volume even when ...


In [93]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub(r'http\S+', '', sent) # remove http
        sent = re.sub(r'https\S+', '', sent) # remove https
        sent = re.sub('<[^>]+>', '', sent) # remove HTML tags
        sent = re.sub('<[^<]+?>', '', sent)
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = re.sub(r'[^\w\s]','',sent) # remove punctuations
        sent = gensim.utils.simple_preprocess(str(sent), min_len=2, deacc=True) 
        
        yield(sent)  

# # Convert to list
data = df.content.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['welcome', 'welcome', 'first', 'ones', 'herethanks', 'was', 'trying', 'to', 'register', 'it', 'but', 'my', 'livestream', 'was', 'delayed', 'may', 'help', 'with', 'the', 'css', 'of', 'this', 'be', 'mod', 'please']]


In [94]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=1,delimiter='_') # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=1, delimiter='_')  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Tag   Meaning                English Examples
# ADJ   adjective              new, good, high, special, big, local
# ADP   adposition             on, of, at, with, by, into, under
# ADV   adverb                 really, already, still, early, now
# CONJ  conjunction            and, or, but, if, while, although
# DET   determiner, article    the, a, some, most, every, no, which
# NOUN  noun                   year, home, costs, time, Africa
# NUM   numeral                twenty-four, fourth, 1991, 14:24
# PRT   particle               at, on, out, over per, that, up, with
# PRON  pronoun                he, their, her, its, my, I, us
# VERB  verb                   is, say, told, given, playing, would
# .     punctuation marks      . , ; !
# X     other                  ersatz, esprit, dunno, gr8, univeristy

# def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
def process_words(texts, stop_words=stop_words, disallowed_postags=['ADP', 'CONJ', 'DET', 'NUM', 'PRT','PRON','.','X']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ not in disallowed_postags])
#         texts_out.append([token.lemma_ for token in doc])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc), max_len=20) if word not in stop_words] for doc in texts_out] 
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!
print(data_ready[:1])

[['welcome', 'welcome', 'first', 'one', 'herethank', 'try', 'register', 'livestream', 'delay', 'help', 'css', 'mod', 'please']]


In [95]:
from gensim.corpora import Dictionary

# Create Dictionary
id2word = Dictionary(data_ready)
print('Number of unique words in initital documents:', len(id2word))

# Filter out words that occur less than 0.5% documents, or more than 20% of the documents.
id2word.filter_extremes(no_below = (round(((len(data_ready))*0.005))), no_above = 0.99)
print('Number of unique words after removing rare and common words:', len(id2word))

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]
print('Number of documents: %d' % len(corpus))

Number of unique words in initital documents: 41672
Number of unique words after removing rare and common words: 1854
Number of documents: 8332


In [96]:
id2word.save("corpus_dict/dict")
corpora.MmCorpus.serialize("corpus_dict/corpus", corpus)

In [97]:
df['tokenz'] = [[(id2word[id]) for id, freq in cp] for cp in corpus[:]]
df.head(1)

,post_id,title,selftext,full_link,author,score,publish_date,num_of_comments,permalink,flair,comment_msg,content,tokenz
0,6fgd2h,Welcome,Welcome!,https://www.reddit.com/r/HomePod/comments/6fgd...,ChicagoIL,12,2017-06-06 03:13:54,7,/r/HomePod/comments/6fgd2h/welcome/,NaN,"['first ones here!,Thanks!\n\nI was trying to ...",welcome welcome first ones herethanks i was tr...,"[delay, first, help, one, please, try, welcome]"


In [98]:
df.to_csv('1_df_content_tokenz.csv',index=False, encoding='utf-8')

In [99]:
print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])

[[('delay', 1), ('first', 1), ('help', 1), ('one', 1), ('please', 1), ('try', 1), ('welcome', 2)]]


In [14]:
# #tf-idf
# from gensim.models import TfidfModel

# # Create Dictionary
# from gensim import models

# tfidf = models.TfidfModel(corpus, id2word=id2word)  # step 1 -- initialize a model
# corpus = tfidf[corpus]
# for doc in corpus:
#     pprint(doc)
#     break

# # print('Number of unique tokens: %d' % len(id2word))
# # print('Number of documents: %d' % len(corpus))

In [15]:
# topWords = {}
# for doc in corpus:
#     for iWord, tf_idf in doc:
#         if iWord not in topWords:
#             topWords[iWord] = 0

#         if tf_idf > topWords[iWord]:
#             topWords[iWord] = tf_idf
# sum = 0
# term = []
# for i, item in enumerate(sorted(topWords.items(), key=lambda x: x[1], reverse=True), 1):
# #     print("%2s: %-13s %s" % (i, id2word[item[0]], item[1]))
#     term.append(id2word[item[0]])
#     sum += item[1]
# #     if i == 100: break
# # print (sum)
# mean = sum/i
# print ('Mean of tf-idf score: ' + str(mean))
# # print (term)

In [16]:
# #tf-idf
# from gensim.models import TfidfModel

# # Create Dictionary
# from gensim import models

# low_value = 0.271734994034526
# low_value_words = []

# tfidf = models.TfidfModel(corpus, id2word=id2word)  # step 1 -- initialize a model
# corpus = tfidf[corpus]
# for doc in corpus:
#     low_value_words += [id for id, value in tfidf[doc] if value < low_value]

In [17]:
# id2word.filter_tokens(bad_ids=low_value_words)
# print('Number of filtered unique tokens: %d' % len(id2word))
# print('Number of documents: %d' % len(corpus))

In [18]:
# corpus = [id2word.doc2bow(doc) for doc in data_ready]
# corpus = tfidf[corpus]
# for doc in corpus:
#     pprint(doc)

In [ ]:
# print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:10]])

In [ ]:
# # Build LDA model
# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=10)

# pprint(lda_model.print_topics())

In [19]:
def compute_coherence_values(corpus, dictionary, num_topics, a, b):
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=num_topics, 
                                                random_state=100,
                                                chunksize=100,
                                                passes=40,
                                                iterations=1000,
                                                alpha=a,
                                                eta=1/num_topics,
                                                eval_every = None)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=df['tokenz'], dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [20]:
# 1

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_1 = pd.DataFrame(model_results)
model_results_1.to_csv('tuning/00_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [21]:
# 2

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_2 = pd.DataFrame(model_results)
model_results_2.to_csv('tuning/11_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [22]:
# 3

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_3 = pd.DataFrame(model_results)
model_results_3.to_csv('tuning/22_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [23]:
# 4

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_4 = pd.DataFrame(model_results)
model_results_4.to_csv('tuning/33_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [24]:
# 5

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_5 = pd.DataFrame(model_results)
model_results_5.to_csv('tuning/44_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [25]:
# 6

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_6 = pd.DataFrame(model_results)
model_results_6.to_csv('tuning/55_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [26]:
# 7

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_7 = pd.DataFrame(model_results)
model_results_7.to_csv('tuning/66_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [27]:
# 8

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_8 = pd.DataFrame(model_results)
model_results_8.to_csv('tuning/77_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [28]:
# 9

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_9 = pd.DataFrame(model_results)
model_results_9.to_csv('tuning/88_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [29]:
# 10

topics_range = [5,10,20,30]

# Alpha parameter
alpha = [.01,.05,.1,.2,.5,1]
# alpha.append('symmetric')
# alpha.append('asymmetric')

# Beta parameter
# beta = [1/num_topics]
# beta.append('symmetric')
# beta.append('auto')

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# iterate through number of topics
for num_topics in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
#         for b in beta:
        # get the coherence score for the given parameters
        cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                      num_topics=num_topics, a=a, b=1/num_topics)
        # Save the model results
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append(a)
        model_results['Beta'].append(1/num_topics)
        model_results['Coherence'].append(cv)
        print (num_topics)
        print (a)
        print (1/num_topics)
        print (cv)

model_results_10 = pd.DataFrame(model_results)
model_results_10.to_csv('tuning/99_lda_tuning_results.csv', index=False, encoding='utf-8')

5
0.01
0.2
0.5188130918322591
5
0.05
0.2
0.545049684376798
5
0.1
0.2
0.5326394508338026
5
0.2
0.2
0.5575651232734606
5
0.5
0.2
0.588301711586334
5
1
0.2
0.5919125569252406
10
0.01
0.1
0.4066444260348726
10
0.05
0.1
0.41093202698511055
10
0.1
0.1
0.4156767486723698
10
0.2
0.1
0.42337417353502893
10
0.5
0.1
0.4671992000397681
10
1
0.1
0.4958397118053225
20
0.01
0.05
0.41178793077960013
20
0.05
0.05
0.4040119407992043
20
0.1
0.05
0.3940310881024052
20
0.2
0.05
0.4526102997870912
20
0.5
0.05
0.4547797939518479
20
1
0.05
0.45941629694362857
30
0.01
0.03333333333333333
0.4765001008831436
30
0.05
0.03333333333333333
0.48352642244669114
30
0.1
0.03333333333333333
0.48551002905783364
30
0.2
0.03333333333333333
0.5121021351758928
30
0.5
0.03333333333333333
0.4194623759371144
30
1
0.03333333333333333
0.398281893352154


In [30]:
model_results = pd.concat([model_results_1, model_results_2, model_results_3, model_results_4, model_results_5,
                          model_results_6, model_results_7, model_results_8, model_results_9, model_results_10])
model_results.to_csv("tuning/model_results.csv", index=False, encoding='utf-8-sig')

In [31]:
model_results = model_results.groupby(['Topics', 'Alpha'], as_index=False).mean()
model_results = model_results.sort_values(by='Coherence', ascending=False)
model_results.to_csv('2_lda_tuning_results.csv', index=False)

In [32]:
model_results

,Topics,Alpha,Beta,Coherence
5,5,1.00,0.200000,0.591913
4,5,0.50,0.200000,0.588302
3,5,0.20,0.200000,0.557565
1,5,0.05,0.200000,0.545050
2,5,0.10,0.200000,0.532639
0,5,0.01,0.200000,0.518813
21,30,0.20,0.033333,0.512102
11,10,1.00,0.100000,0.495840
20,30,0.10,0.033333,0.485510
19,30,0.05,0.033333,0.483526


In [ ]:
# priors = pd.pivot_table(model_results,index=["Topics"],columns=["Alpha"],values=['Coherence'])
# priors.columns = range(priors.shape[1])
# priors.columns = ['.01','.05','.1','.2','.5','1']
# df.head(1)
# priors = priors.reset_index()
# priors

In [ ]:
# priors.to_csv("siri_lda_tuning_results.csv",index=True, encoding="utf-8")

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
  
# # dummy data
# x1 = priors['Topics']
# A = priors['.01']
# B = priors['.05']
# C = priors['.1']
# D = priors['.2']
# E = priors['.5']
# F = priors['1']

# # creates two subplots
# # fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (24, 12))

# fig, ax = plt.subplots(2, 3, figsize = (24,12))

# # Plot without grid
# ax[0,0].plot(x1, A, label='0.01', color='tab:blue')
# ax[0,1].plot(x1, B, label='0.05', color='tab:orange')
# ax[0,2].plot(x1, C, label='0.1', color='tab:green')
# ax[1,0].plot(x1, D, label='0.2', color='tab:red')
# ax[1,1].plot(x1, E, label='0.5', color='tab:purple')
# ax[1,2].plot(x1, F, label='1', color='tab:brown')

# ax[0,0].set_xlim(xmin=9)
# ax[0,0].set_title('siri, α=.01, Beta=1/K')
# ax[0,0].set_xlabel('K')
# ax[0,0].set_ylabel('Cv')

# ax[0,1].set_xlim(xmin=9)
# ax[0,1].set_title('siri, α=.05, Beta=1/K')
# ax[0,1].set_xlabel('K')
# ax[0,1].set_ylabel('Cv')

# ax[0,2].set_xlim(xmin=9)
# ax[0,2].set_title('siri, α=.1, Beta=1/K')
# ax[0,2].set_xlabel('K')
# ax[0,2].set_ylabel('Cv')

# ax[1,0].set_xlim(xmin=9)
# ax[1,0].set_title('siri, α=.2, Beta=1/K')
# ax[1,0].set_xlabel('K')
# ax[1,0].set_ylabel('Cv')

# ax[1,1].set_xlim(xmin=9)
# ax[1,1].set_title('siri, α=.5, Beta=1/K')
# ax[1,1].set_xlabel('K')
# ax[1,1].set_ylabel('Cv')

# ax[1,2].set_xlim(xmin=9)
# ax[1,2].set_title('siri, α=1, Beta=1/K')
# ax[1,2].set_xlabel('K')
# ax[1,2].set_ylabel('Cv')

# # fig.tight_layout()
# fig.set_facecolor("w")
# plt.show()